In [1]:
import scipy.io.wavfile as wf
import numpy as np
import matplotlib.pyplot as mp
import wave as wav
import os
import librosa
import soundfile
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
import tensorflow as tf
import generate_masking_threshold as generate_mask
import base64
import hashlib
import json 
import time
import uuid
import requests
import scipy
import random
import talentedsoft
#import scikits.audiolab

def compute_mfcc(audio, **kwargs):
    """
    Compute the MFCC for a given audio waveform. This is
    identical to how DeepSpeech does it, but does it all in
    TensorFlow so that we can differentiate through it.
    """
    batch_size, size = audio.get_shape().as_list()
    audio = tf.cast(audio, tf.float32)

    # 1. Pre-emphasizer, a high-pass filter
    audio = tf.concat(
        (audio[:, :1], audio[:, 1:] - 0.97 * audio[:, :-1], np.zeros((batch_size, 512), dtype=np.float32)), 1)

    # 2. windowing into frames of 512 samples, overlapping
    windowed = tf.stack([audio[:, i:i + 512] for i in range(0, size - 320, 320)], 1)

    window = np.hamming(512)
    windowed = windowed * window

    # 3. Take the FFT to convert to frequency space
    ffted = tf.spectral.rfft(windowed, [512])
    ffted = 1.0 / 512 * tf.square(tf.abs(ffted))

    # 4. Compute the Mel windowing of the FFT
    energy = tf.reduce_sum(ffted, axis=2) + np.finfo(float).eps
    filters = np.load("filterbanks.npy").T
    feat = tf.matmul(ffted, np.array([filters] * batch_size, dtype=np.float32)) + np.finfo(float).eps

    # 5. Take the DCT again, because why not
    feat = tf.log(feat)
    feat = tf.spectral.dct(feat, type=2, norm='ortho')[:, :, :26]

    # 6. Amplify high frequencies for some reason
    _, nframes, ncoeff = feat.get_shape().as_list()
    n = np.arange(ncoeff)
    lift = 1 + (22 / 2.) * np.sin(np.pi * n / 22)
    feat = lift * feat
    width = feat.get_shape().as_list()[1]

    # 7. And now stick the energy next to the features
    feat = tf.concat((tf.reshape(tf.log(energy), (-1, width, 1)), feat[:, :, 1:]), axis=2)
    return feat

def pad(mfcc2,mfcc1):
    size1 = mfcc1.get_shape().as_list()
    size2 = mfcc2.get_shape().as_list()
    length = size1[1] - size2[1]
    paddings = [[0,0],[0,length],[0,0]]
    return tf.pad(mfcc2,paddings,"CONSTANT")

def SNR(origanl, current):
    noise=current-origanl
    origanl_ar = sess.run(origanl)
    noise_ar = sess.run(noise)
    ans = 20*np.log10(np.linalg.norm(origanl_ar, ord=2)/np.linalg.norm(noise_ar, ord=2))
    return ans

def new_input(noise, audio_sequence1):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tensor = tf.clip_by_value(noise+audio_sequence1, -2**15, 2**15-1)
        array = sess.run(tensor)
    return array

def psychoacoustic_loss(adv_audio, original, psd_max_a, th,transform):
    logits_delta = transform((adv_audio[0,:] - original[0,:]),psd_max_a[0])
    psychoacoustic_loss = tf.reduce_mean(tf.nn.relu(logits_delta - th[0]))
    return psychoacoustic_loss

def truncate(mfcc1, mfcc2):
    size1 = mfcc1.get_shape().as_list()#origin
    #size2 = mfcc2.get_shape().as_list()#target
    mfcc2 = tf.slice(mfcc2,[0,0,0],[1,size1[1],26])
    return mfcc2

def loss(original, target, noise, psychoacoustic_loss,alpha):
    mfcc1 = compute_mfcc(original + noise)
    size1 = original.get_shape().as_list()
    size2 = target.get_shape().as_list()
    if size1[1] > size2[1]:
        mfcc2 = pad(compute_mfcc(target),mfcc1)
    else:
        mfcc2 = truncate(mfcc1,compute_mfcc(target))
    loss = tf.norm(mfcc1-mfcc2,2) + alpha * psychoacoustic_loss
    #+ 0.01 * tf.cast(tf.norm(noise,2),dtype = tf.float32)
#     loss =  tf.norm(cal_mfcc(new_input(noise, audio_sequence1),sample_rate1) 
#     - pad(cal_mfcc(audio_sequence2, sample_rate2)),2) + 0.001 * tf.cast(tf.norm(noise, 2),dtype = tf.float64)
    return loss
APPId = "fcfefe42"
APISecret = "NmE5NTZkNzFlMzg3MGM0ZDBmZTdkODYx"
APIKey = "aba4d1c2815577b411d5d5a4966796aa"
class Gen_req_url(object):
    """生成请求的url"""

    def sha256base64(self, data):
        sha256 = hashlib.sha256()
        sha256.update(data)
        digest = base64.b64encode(sha256.digest()).decode(encoding='utf-8')
        return digest

    def parse_url(self, requset_url):
        stidx = requset_url.index("://")
        host = requset_url[stidx + 3:]
        # self.schema = requset_url[:stidx + 3]
        edidx = host.index("/")
        if edidx <= 0:
            raise Exception("invalid request url:" + requset_url)
        self.path = host[edidx:]
        self.host = host[:edidx]

    # build websocket auth request url
    def assemble_ws_auth_url(self, requset_url, api_key, api_secret, method="GET"):
        self.parse_url(requset_url)
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))
        # date = "Thu, 12 Dec 2019 01:57:27 GMT"
        signature_origin = "host: {}\ndate: {}\n{} {} HTTP/1.1".format(self.host, date, method, self.path)
        signature_sha = hmac.new(api_secret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')
        authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
            api_key, "hmac-sha256", "host date request-line", signature_sha)
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
        values = {
            "host": self.host,
            "date": date,
            "authorization": authorization
        }

        return requset_url + "?" + urlencode(values)


def gen_req_body(apiname, APPId, file_path=None):
    """
    生成请求的body
    :param apiname
    :param APPId: Appid
    :param file_name:  文件路径
    :return:
    """
    if apiname == 'createFeature':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createFeature",
                    "groupId": "8",
                    "featureId": "cmu_scottish",
                    "featureInfo": "wav08",
                    "createFeatureRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'createGroup':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "createGroup",
                    "groupId": "8",
                    "groupName": "cmu_dataset",
                    "groupInfo": "cmu_dataset",
                    "createGroupRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
   
    elif apiname == 'queryFeatureList':

        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "queryFeatureList",
                    "groupId": "8",
                    "queryFeatureListRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            }
        }
    elif apiname == 'searchFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchFea",
                    "groupId": "8",
                    "topK": 10,
                    "searchFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    elif apiname == 'searchScoreFea':

        with open(file_path, "rb") as f:
            audioBytes = f.read()
        body = {
            "header": {
                "app_id": APPId,
                "status": 3
            },
            "parameter": {
                "s782b4996": {
                    "func": "searchScoreFea",
                    "groupId": "iFLYTEK_examples_groupId",
                    "dstFeatureId": "p374",
                    "searchScoreFeaRes": {
                        "encoding": "utf8",
                        "compress": "raw",
                        "format": "json"
                    }
                }
            },
            "payload": {
                "resource": {
                    "encoding": "lame",
                    "sample_rate": 16000,
                    "channels": 1,
                    "bit_depth": 16,
                    "status": 3,
                    "audio": str(base64.b64encode(audioBytes), 'UTF-8')
                }
            }
        }
    else:
        raise Exception(
            "输入的apiname不在[createFeature, createGroup, deleteFeature, queryFeatureList, searchFea, searchScoreFea,updateFeature]内，请检查")
    return body

def req_url(api_name, APPId, APIKey, APISecret, file_path=None):
    """
    开始请求
    :param APPId: APPID
    :param APIKey:  APIKEY
    :param APISecret: APISecret
    :param file_path: body里的文件路径
    :return:
    """
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    #print(tempResult)
    #print(base64.b64decode(tempResult['payload'][api_name + 'Res']['text']))
"""
 * 1.声纹识别接口,请填写在讯飞开放平台-控制台-对应能力页面获取的APPID、APIKey、APISecret
 * 2.groupId要先创建,然后再在createFeature里使用,不然会报错23005,修改时需要注意保持统一
 * 3.音频base64编码后数据(不超过4M),音频格式需要16K、16BIT的MP3音频。
 * 4.主函数只提供调用示例,其他参数请到对应类去更改,以适应实际的应用场景。
"""
def req_score(api_name, APPId,  APIKey, APISecret, file_path=None):
    gen_req_url = Gen_req_url()
    body = gen_req_body(apiname=api_name, APPId=APPId, file_path=file_path)
    request_url = gen_req_url.assemble_ws_auth_url(requset_url='https://api.xf-yun.com/v1/private/s782b4996', method="POST", api_key=APIKey, api_secret=APISecret)
    headers = {'content-type': "application/json", 'host': 'api.xf-yun.com', 'appid': '$APPID'}
    response = requests.post(request_url, data=json.dumps(body), headers=headers)
    tempResult = json.loads(response.content.decode('utf-8'))
    #print(tempResult)
    return base64.b64decode(tempResult['payload'][api_name + 'Res']['text'])#,base64.b64decode(tempResult['payload'][api_name + 'Res']['text']['scoreList'][0]['score'])


def GetSpeakerScore(speaker,value):
    m = len(value)
    result = ''
    for i in range(0,m):
        result+=chr(value[i]) #switch to string
    result = eval(result)
    v = list(result.values())[0]
    #print(v)
    lenv=len(v)
    return v[0]['featureId'], v[0]['score'], v[1]['featureId'],v[1]['score']    
# def getSign(timestamp, nonce):
#     hs = hashlib.sha256()
#     appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
#     secret = '3c8a61e2bdb0ee2d0af74814142ba2ee'
#     hs.update((appkey + timestamp + secret + nonce).encode('utf-8'))
#     signature = hs.hexdigest().upper()
#     return signature
# def identifyFeatureByGroupId(confirmFeatureFileName):
#     identify_feature = open(confirmFeatureFileName, 'rb').read()
#     # 声纹base64字符串
#     audio_data = base64.b64encode(identify_feature)
#     timestamp = str(int(time.time() * 1000))
#     nonce = str(uuid.uuid1()).replace('-', '')
#     sign = getSign(timestamp, nonce)
#     headers = {"Content-Type": "application/json"}
#     appkey = "zvpcvm5hxib3jz3vt2jshqxjndcywwo2qnx7s6iy"
#     groupId = '15'
#     host = 'https://ai-vpr.hivoice.cn'
#     identifyFeatureByGroupIdEndPoint = '/vpr/v1/identifyFeatureByGroupId'
#     identify_feature_param = {
#         "appkey": appkey,
#         "timestamp": timestamp,
#         "nonce": nonce,
#         "sign": sign,
#         "groupId": groupId,
#         "topN": 6,
#         "audioData": audio_data.decode(),
#         "audioSampleRate": 16000,
#         "audioFormat": "wav"
#     }
#     #print('identify_feature_param', identify_feature_param)
#     identify_feature_resp = requests.post(url=host + identifyFeatureByGroupIdEndPoint,
#                                           data=json.dumps(identify_feature_param),
#                                           headers=headers)
#     identify_feature_result = json.loads(identify_feature_resp.content)
#     return identify_feature_result['data'][0]['featureInfo'], identify_feature_result['data'][0]['score'],identify_feature_result['data'][1]['featureInfo'], identify_feature_result['data'][1]['score']
def trans_wav_mp3(file_path):
    """
    将wav文件转为mp3文件，保存到cmu_mp3
    """
    sourcefile = AudioSegment.from_wav(file_path)
    #filename = file_path.split('/')[-1].split('.wav')[0].replace(' ', '_') + '.mp3'
    filename = file_path.split('/')[-1].split('.wav')[0] + '.mp3'
    save_path = "/mnt/data/Chenpinji/mfcc_advGen/mp3/"
    #print(filename)
    sourcefile.export(save_path + filename, format="mp3")

class Transform(object):
    '''
    Return: PSD
    '''
    def __init__(self, window_size):
        self.scale = 8. / 3.
        self.frame_length = int(window_size)
        self.frame_step = int(window_size // 4)
        self.window_size = window_size

    def __call__(self, x, psd_max_ori):
        win = tf.contrib.signal.stft(x, self.frame_length, self.frame_step)
        z = self.scale * tf.abs(win / self.window_size)
        psd = tf.square(z)
        PSD = tf.pow(10., 9.6) / tf.reshape(psd_max_ori, [-1, 1, 1]) * psd
        return PSD

In [7]:

filename1 = '/mnt/data/Chenpinji/cmu_dataset/cmu1/wav/arctic_a0006.wav'
filename2 = '/mnt/data/Chenpinji/cmu_dataset/cmu2/wav/arctic_a0008.wav'
sample_rate1, audio_sequence1 = wf.read(filename1)
sample_rate2, audio_sequence2 = wf.read(filename2)
#time = a1 / f1#音频时长
audio1 = np.expand_dims(audio_sequence1,0)
audio2 = np.expand_dims(audio_sequence2,0)
len1 = len(audio1[0])
len2 = len(audio2[0])
th, psd_max = generate_mask.generate_th(audio_sequence1.astype(float), 16000, 2048)
th=np.expand_dims(th,0)
psd_max=np.expand_dims(psd_max,0)
transform = Transform(2048)
psd_max_a = tf.cast(psd_max, tf.float32)

#stage 1, nearly no constraint on noise size, which aims to make the attack succeed
original = tf.Variable(np.zeros((1, len1), dtype=np.float32))
target  = tf.Variable(np.zeros((1, len2), dtype=np.float32))
with tf.variable_scope("noise"):
    noise=tf.Variable(tf.random_normal([1,len1],stddev = 50))
    #noise=tf.clip_by_value(noise,-5000,5000)
    t_vars = tf.trainable_variables()
    noise_vars = [var for var in t_vars if "noise" in var.name]
#config = tf.ConfigProto(device_count = {'CPU': 4})
with tf.Session() as sess:
    train_teacher = tf.train.AdamOptimizer(2).minimize (loss(original, target, noise, psychoacoustic_loss(original + noise, original, psd_max_a, th,transform),0.00001), var_list=noise_vars)
    sess.run(tf.global_variables_initializer())
    original = original.assign(np.array(audio1))
    target   = target.assign(np.array(audio2))
    select3 = 2000
    for i in range(select3):
        sess.run((train_teacher))
        if i % 100 == 0 or i == select3 - 1:
            a,b,NewInput= sess.run([loss(original, target, noise, psychoacoustic_loss(original + noise, original, psd_max_a, th,transform), 0.001),psychoacoustic_loss(original + noise, original, psd_max_a, th,transform), original + noise])
            print(i)
            snr = SNR(original, NewInput)
            print('current loss:',a,'SNR:',snr)
            print('current psychoacoustic_loss', b)
            oname = 'cmu1_cmu2_'+str(snr)
            audio_name = '/mnt/data/Chenpinji/mfcc_advGen/'+ oname + '.wav'
            scaled = np.array(np.clip(np.round(NewInput[0]),-2 ** 15, 2 ** 15 - 1),dtype = np.int16)
            wf.write(audio_name, 16000 ,scaled)
            #trans_wav_mp3(audio_name)
            #value = req_score(api_name='searchFea', APPId=APPId,APIKey=APIKey, APISecret=APISecret, file_path='/mnt/data/Chenpinji/adv_mfccGen/mp3'+ oname + '.mp3')
            #speaker1,score1,speaker2,score2 = GetSpeakerScore('cmu7',value)
            asperson, s = talentedsoft.talent_score(audio_name,'cmu7')
            print('classified as:',asperson,'Score is:',s)
            #print('second place is:',speaker2, 'Score is:',score2)
        
        

0
current loss: 1016.82965 SNR: 34.31811332702637
current psychoacoustic_loss 1588.2177
classified as: cmu1 Score is: 0.985
100
current loss: 747.7936 SNR: 27.695353031158447
current psychoacoustic_loss 9464.327
classified as: cmu1 Score is: 0.943
200
current loss: 678.8578 SNR: 23.566372394561768
current psychoacoustic_loss 26116.97
classified as: cmu1 Score is: 0.78
300
current loss: 647.3849 SNR: 20.9637451171875
current psychoacoustic_loss 48965.668
classified as: cmu1 Score is: 0.529
400
current loss: 633.7998 SNR: 19.069232940673828
current psychoacoustic_loss 76691.53
classified as: cmu1 Score is: 0.298
500
current loss: 632.40515 SNR: 17.59600043296814
current psychoacoustic_loss 108270.305
classified as: cmu2 Score is: 0.178
600
current loss: 640.3434 SNR: 16.395325660705566
current psychoacoustic_loss 143002.22
classified as: cmu2 Score is: 0.203


KeyboardInterrupt: 